In [146]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from pathlib import Path
import os
import nltk

#nltk.download('stopwords')

basepath = os.path.dirname(os.path.abspath('__file__'))
datapath1 = os.path.dirname(basepath)
datapath = os.path.dirname(datapath1)

np.random.seed(500)

In [147]:
#datapath = Path(__file__).resolve().parents[2]

part = "part1"
readfile = datapath + '/results/hugh-murray/part1/processed/part1-annotated-special.csv'
data1 = pd.read_csv(readfile, sep='\t', encoding='latin1', error_bad_lines=False)

In [148]:
data = data1[data1.Location.notnull()].reset_index()

print("test")
#data['sentence1']= [word_tokenize(entry) for entry in data['sentence']]

# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,row in data.iterrows():
    
    if row['Location'] in row['sectionTitle']:
        is_in_section = 1
    else:
        is_in_section = 0
        
    #print(type(row['Travel verbs']))
    if not isinstance(row['Travel verbs'],float):
        is_travel_verb = 1
    else:
        is_travel_verb = 0
    
    entry =  word_tokenize(row['sentence'])
    
    if not isinstance(row['Travel Noun Phrases'],float):
        if row['Location'] in row['Travel Noun Phrases']:
            is_in_travel_phrase = 1
        else:
            is_in_travel_phrase = 0
    else:
        is_in_travel_phrase = 0
    
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    #print(str(Final_words))
    data.loc[index,'isInSection'] = int(is_in_section)
    data.loc[index,'isTravelVerb'] = int(is_travel_verb)
    data.loc[index,'isInTravelPhrase'] = int(is_in_travel_phrase)
    data.loc[index,'sentence_final'] = str(Final_words)  

#column_trans = ColumnTransformer(
#    [('section_title', OneHotEncoder(dtype='int'),['sectionTitle']),
#      ('sentence', TfidfVectorizer(), 'sentence_final')],
#     remainder='drop')


features = data[['isInSection','isTravelVerb','isInTravelPhrase','sentence_final']].copy()
#features = data[['isInSection','isTravelVerb','isInTravelPhrase','sentence_final']].copy()

print(features.dtypes)
target = data.Result.values
#print(features)    
print("Okay Done")    

test
isInSection         float64
isTravelVerb        float64
isInTravelPhrase    float64
sentence_final       object
dtype: object
Okay Done


In [180]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(features,target,test_size=0.3)

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
print("done")

done


In [181]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
numerical_features = features.dtypes == 'float'
#model = naive_bayes.MultinomialNB()
model = SVC(gamma='auto')
#transformer = ColumnTransformer(transformers=[('isInSection', OneHotEncoder(), [0, 1])])

transformer = ColumnTransformer(transformers=[('sentence_tfidf', TfidfVectorizer(), 'sentence_final')])


pipe = Pipeline(steps=[('t', transformer), ('m',model)])

#model.fit_transform(features)

pipe.fit(Train_X, Train_Y)
print("Multinomial NB score: %f" % pipe.score(Test_X, Test_Y))







Multinomial NB score: 0.433333


## 

In [48]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  66.66666666666666


In [ ]:
>